In [ ]:
%load_ext autoreload
%autoreload 2

import keras

import numpy as np
from keras.models import Model, load_model, Sequential
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import Adam
from keras.objectives import categorical_crossentropy

import matplotlib.pyplot as plt
import glob
import os
from sklearn.externals import joblib
import sys

sys.path.append('../')

import paths

%matplotlib inline



img_size = 256
batch_size = 16

In [ ]:
folder_name = 'test-jpg'

def preprocess_gen(gen):
    for X, y in gen:
        yield preprocess_input(X)/255., y
        
data_gen = ImageDataGenerator().flow_from_directory(paths.DATA_FOLDER, batch_size=batch_size, shuffle=False,
                                                    classes=[folder_name], target_size=(img_size, img_size))
n_files = data_gen.n
data_gen = preprocess_gen(data_gen)

In [ ]:
base_model = VGG16(include_top=False, input_shape=(3, img_size, img_size))
for layer in base_model.layers:
    layer.trainable = False
output = GlobalAveragePooling2D()(base_model.output)
model = Model(input=base_model.input, output=output)
model.compile(Adam(lr=1e-4), 'categorical_crossentropy')
model.summary()

In [ ]:
num_steps = (n_files / batch_size) + 1
preds = model.predict_generator(data_gen, val_samples=n_files)

In [ ]:
preds = preds[:n_files, :]

In [ ]:
preds.shape

In [ ]:
joblib.dump(preds, os.path.join(paths.DATA_FOLDER, 'vgg_glob_avg_pool_features_test.pkl'))